In [1]:
import numpy as np

from lib import *

In [22]:
@level_ab(1, test=('1000\n2000\n3000\n\n4000\n\n5000\n6000\n\n7000\n8000\n9000\n\n10000', 24000, 45000), sep="\n\n")
def solve(num: List[str], level=0):
    chunks = [sum(map(int, line.split("\n"))) for line in num]
    return sum(list(sorted(chunks, reverse=True))[:3]) if level else max(chunks)

Part a already solved with same answer: 69912
Part b already solved with same answer: 208180


In [19]:
@level_ab(2, test=("""A Y\nB X\nC Z""", 15, 12), sep="\n", apply=lambda line: line.split(" "))
def solve(lines: List[str], level=0):
    total = 0
    for a, b in lines:
        a, b = ord(a) - ord('A'), ord(b) - ord('X')
        total += b * 3 + (a+b-1) % 3 + 1 if level else ((b-a+1)%3) * 3 + b + 1
    return total

Part a already solved with same answer: 11603
Part b already solved with same answer: 12725


In [28]:
def prio(c: str) -> int:
    return ord(c.lower()) - ord('a') + 1 + c.isupper() * 26

@level_ab(3, test=("""vJrwpWtwJgWrhcsFMMfFFhFp\njqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL\nPmmdzqPrVvPwwTWBwg\nwMqvLMZHhHMvwLHjbvcjnnSBnvTQFn\nttgJtRGJQctTZtZT\nCrZsJsPPZsGzwwsLwLmpwMDw""", 157, 70), sep="\n")
def solve(lines: List[str], level=0):
    total = 0
    if level:
        for i in range(0, len(lines)-1, 3):
            a, b, c = map(set, lines[i: i+3])
            total += prio(next(iter(a & b & c)))
    else:
        for line in lines:
            offset = len(line)//2
            a, b = set(line[:offset]), set(line[offset:])
            total += sum([prio(c) for c in a & b])
    return total

Part a already solved with same answer: 8176
Part b already solved with same answer: 2689


In [20]:
@level_ab(4, test=("""2-4,6-8\n2-3,4-5\n5-7,7-9\n2-8,3-7\n6-6,4-6\n2-6,4-8""", 2, 4), sep="\n",
          apply=lambda line: map(int, re.findall(r"(\d+)-(\d+),(\d+)-(\d+)", line)[0]))
def solve(lines: List[str], level=0, ):
    total = 0
    for a, b, c, d in lines:
        left, right = set(range(a, b+1)), set(range(c, d+1))
        total += bool(left & right) if level else left.issuperset(right) or right.issuperset(left)
    return total

Part a already solved with same answer: 584
Part b already solved with same answer: 933


In [3]:
@level_ab(5, test=('    [D]\n[N] [C]\n[Z] [M] [P]\n 1   2   3\n\nmove 1 from 2 to 1\nmove 3 from 1 to 3\nmove 2 from 2 to 1\nmove 1 from 1 to 2', "CMZ", "MCD"), sep="\n")
def solve(lines: List[str], level=0):
    stacks = defaultdict(list)
    lines = iter(lines)
    for line in lines:
        if line.startswith(" 1 "):
            next(lines)
            break
        for pos, dst in enumerate(line):
            if dst.isalpha():  # skip "[", "]", " "
                stacks[1+pos//4].insert(0, dst)
    for line in lines:
        _, amount, _, src, _, dst = line.split(" ")
        amount, src, dst = int(amount), int(src), int(dst)
        picked = [stacks[src].pop() for _ in range(amount)]
        stacks[dst].extend(reversed(picked) if level else picked)
    return "".join([stacks[i][-1] for i in sorted(stacks.keys())])

Part a already solved with same answer: TPGVQPFDH
Part b already solved with same answer: DMRDFRHHH


In [11]:
@level_ab(6, test=("""mjqjpqmgbljsphdztnvjfqwrcgsmlb""", 7, 19), sep="\n")
def solve(lines: List[str], level=0):
    offset = 4 + level * 10
    for line in lines:
        for i in range(len(line)-offset):
            if len(set(line[i:i+offset])) == offset:
                return i+offset

Part a already solved with same answer: 1134
Part b already solved with same answer: 2263


In [39]:
def dir_sum(dir_name: str, root: Dict[str, Dict], dirs_name_size: Dict[str, int]) -> int:
    """ recursively sum up sizes of subdirectories and add all to a global list """
    total_size = 0
    for file_name, file_dict_or_size in root.items():
        if type(file_dict_or_size) == dict and file_name != "..":
            total_size += dir_sum(dir_name + "/" + file_name, file_dict_or_size, dirs_name_size)
        elif type(file_dict_or_size) == int:
            total_size += file_dict_or_size
    dirs_name_size[dir_name] = total_size
    return total_size

@level_ab(7, test=('$ cd /\n$ ls\ndir a\n14848514 b.txt\n8504156 c.dat\ndir d\n$ cd a\n$ ls\ndir e\n29116 f\n2557 g\n62596 h.lst\n$ cd e\n$ ls\n584 i\n$ cd ..\n$ cd ..\n$ cd d\n$ ls\n4060174 j\n8033020 d.log\n5626152 d.ext\n7214296 k', 95437, 24933642), sep="\n")
def solve(lines: List[str], level=0):
    root = curr = dict()
    for line in lines:
        a, b, *c = line.split(" ")
        if a == "$":
            if b == "cd":  # just ignore ls case
                c = c[0]
                curr = root if c == "/" else curr[c]
        else:
            curr[b] = {"..": curr} if a == "dir" else int(a)
    total = dir_sum("", root, dirs_name_size := dict())
    level_filter = [lambda s: s < 100_000, lambda s: s > total - 40000000][level]
    sizes = sorted(filter(level_filter, dirs_name_size.values()))
    return min(sizes) if level else sum(sizes)


Part a already solved with same answer: 1989474
Part b already solved with same answer: 1111607


In [103]:
@level_ab(8, test=('30373\n25512\n65332\n33549\n35390', 21, 8), sep="\n")
def solve(lines: List[str], level=0):
    f = np.array([[int(c) for c in line] for line in lines])
    vis, score = np.zeros_like(f), np.ones_like(f, dtype=int)
    for _ in range(4):  # repeat with 4 rotations
        for (i, j), val in np.ndenumerate(f):
            if level:  # walk down 1, 2, 3... steps
                for d in range(0, len(f)-j):
                    if d == 0:
                        continue
                    if f[i, j+d] >= f[i, j]:
                        break
                score[i, j] *= d
            else:
                if j == 0 or max(f[i, :j]) < f[i, j]:
                    vis[i, j] = True
        # do again with rot90 field
        f, vis, score = np.rot90(f), np.rot90(vis), np.rot90(score)
    return score.max() if level else vis.sum()

Part a already solved with same answer: 1662
Part b already solved with same answer: 537600


In [ ]:
@level_ab(9, test=("""
""", 1, 2), sep="\n")
def solve(lines: List[str], level=0):
    f = np.array([[int(c) for c in line] for line in lines])